# OpenRouteService 

We'll be learning how to use this API with `Folium` to generate a map with some coordinates + directions using **ors**

## Folium:

`pip install folium`

Check [folium](https://python-visualization.github.io/folium/latest/getting_started.html) documentation

## ORS:

`pip install openrouteservice`

Check [ors](https://github.com/GIScience/openrouteservice-py) github

Testing we'll run locally but transfer to our API Key once we know what we're doing:
`client = openrouteservice.Client(base_url='http://localhost/ors')`

## Geopy:

`pip install geopy`

Helps us find the long lat we need based on location.
[python package](https://pypi.org/project/geopy/)

# Let's start

[Hosting ORS on Docker (Locally) ](https://www.youtube.com/watch?v=VQXlbqKArFk)
Follow thier [local hosting guide](https://github.com/GIScience/openrouteservice)

Git Clone Openservice:
*Make sure you CD to the right workspace*
*z:* to get into your z drive or any letter drive....
`git clone https://github.com/GIScience/openrouteservice.git`

Now that we're in the *openrouteservice*
We need to make some directories:
`mkdir -p ors-docker/config ors-docker/elevation_cache ors-docker/graphs ors-docker/files ors-docker/logs` or
```
mkdir ors-docker\config
mkdir ors-docker\elevation_cache
mkdir ors-docker\graphs
mkdir ors-docker\files
mkdir ors-docker\logs
```

Use Docker Compose:

Start in background:
`docker compose up -d`

View logs:
`docker compose logs`

Stop Container:
`docker compose down`

Testing: 

[ORS Directions API](https://openrouteservice.org/dev/#/api-docs/v2/directions/)

Running into some errors: We'll just use an API KEY to test :(


In [15]:
import openrouteservice as ors 
import dotenv
import os
import folium

dotenv.load_dotenv()
client = ors.Client(key=os.getenv('ORS_API_KEY')) # Specify your personal API key

# Long Lat
# In google maps we have Lat, Long and Folium does work with lat long however ORS doesn't 
tekbasic_starting_point_coords = [40.41706977125244, -74.52976762042528]

# Generating a map 
m = folium.Map(location=tekbasic_starting_point_coords, tiles="cartodbpositron", zoom_start=13)
m

In [ ]:
from geopy.geocoders import Nominatim
import time
# Scenario --> I want to deliver some product to Petco in New York Larchmont 
# We need to find the Long,Lat for this place but remember Folium uses Lat,Long

address = '1275 W Boston Post Rd, Larchmont, NY 10538'
# To find the long lat of a place we need: geopy
geolocator = Nominatim(user_agent="lms_app")
location = geolocator.geocode(address)
loc_coord = []
if location:
    # Nominatim uses OpenStreetMap API which could block our service if we abuse it 
    # One of their condition is 1 request per second so we'll use sleep here
    time.sleep(1)
    # Long Lat for ORS Route 
    loc_coord = [location.longitude, location.latitude]


In [25]:
# After using geopy to find long lat 
# Building our list of coords to find find direction and build a route with ors 

# Long Lat format to use ors (We'll reverse once we build our map)
coords = [list(reversed(tekbasic_starting_point_coords)), loc_coord]
print(coords)

# Building our route: based on Long Lat 
route = client.directions(
    coordinates=coords,
    profile='driving-car',
    format='geojson'    # Must use geojson to get numbered coordinates 
)

# Mapping route on folium 
# When we map our route, we need to reverse our coordinates to fit the: Lat, Long format
folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']], color='blue').add_to(m)
m

[[-74.52976762042528, 40.41706977125244], [-73.74584238431478, 40.937444400000004]]


In [28]:
# Delivery Times
seconds =route['features'][0]['properties']['summary']['duration']    # Seconds
meter = route['features'][0]['properties']['summary']['distance']   # 1 Mile is 1609 Meters so:

def get_miles(meter):
    return meter/1609

print(get_miles(meter))

def get_time(seconds):
    return time.strftime('%H:%M:%S', time.gmtime(seconds))

print(get_time(seconds))

65.79422001243007
01:39:52
